In [ ]:
# ==========================================================
# import libraries
# ==========================================================
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time
import os


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

LOG_DIR = f"{int(time.time())}"

In [ ]:
# ==========================================================
# define functions
# ==========================================================
def build_model(hp):
    input_layer = Input(shape=(data_x_train_scaled.shape[1]), name='Input_Layer')
    common_path = Dense(units=hp.Int(f"dense_1_cp", 16, 128, 16), activation='relu', name='first_dense')(input_layer)
    common_path = Dropout(hp.Choice('drouput_rate_1', values=[0., 0.1, 0.2, 0.3]))(common_path)
    common_path = Dense(units=hp.Int(f"dense_2_cp", 16, 128, 16), activation='relu', name='second_dense')(common_path)
    common_path = Dropout(hp.Choice('drouput_rate_2', values=[0., 0.1, 0.2, 0.3]))(common_path)

    # path 1
    first_output = Dense(units='1', name='first_output_last_layer')(common_path)
    
    # for the number of hidden layers --> needs to be debugged cause sometimes I got nan as loss
    # first_output_path = Dense(units=hp.Int(f"dense_0_p1", 16, 128, 16), activation='relu',
    #                          name='first_output_dense0')(common_path)
    # for i in range(hp.Int('n_layers_p1', 1, 4)):
    #    first_output_path = Dense(units=hp.Int(f"dense_{i+1}_p1_", 16, 128, 16), activation='relu')(first_output_path)
    # first_output = Dense(units='1', name='first_output_last_layer')(first_output_path)

    # path 2
    second_output_path = Dense(units=hp.Int(f"dense_1_p2", 16, 128, 16), activation='relu',
                               name='second_output_dense0')(common_path)
    second_output_path = Dropout(hp.Choice('drouput_rate_3', values=[0., 0.1, 0.2, 0.3]))(second_output_path)
    second_output = Dense(units='1', name='second_output_last_layer')(second_output_path)
    #  for the number of hidden layers --> needs to be debugged cause sometimes I got nan as loss
    # second_output_path = Dense(units=hp.Int(f"dense_0_p2", 16, 128, 16), activation='relu',
    #                           name='second_output_dense0')(common_path)
    # for i in range(hp.Int('n_layers_p2', 1, 4)):
    #     second_output_path = Dense(units=hp.Int(f"dense_{i+1}_p2_", 16, 128, 16), activation='relu')(second_output_path)
    # second_output = Dense(units='1', name='second_output_last_layer')(second_output_path)

    #
    model = Model(inputs=input_layer, outputs=[first_output, second_output])
    
    model.compile(optimizer=tf.keras.optimizers.SGD(hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
                  loss={'first_output_last_layer': 'mse', 'second_output_last_layer': 'mse'},
                  metrics={'first_output_last_layer': tf.keras.metrics.RootMeanSquaredError(),
                           'second_output_last_layer': tf.keras.metrics.RootMeanSquaredError()})
    return model

y1 Heating Load

y2 Cooling Load

In [ ]:
# ==========================================================
# data preparation
# ==========================================================
data = pd.read_csv('../input/eergy-efficiency-dataset/ENB2012_data.csv')
data_x = data.iloc[:, :-2]
data_y = data.iloc[:, -2:]

del data

#
data_x_train_scaled, data_x_test_scaled, data_y_train, data_y_test = \
    train_test_split(data_x, data_y, test_size=0.2, random_state=42)

#
sc = StandardScaler()
data_x_train_scaled = sc.fit_transform(data_x_train_scaled)
data_x_test_scaled = sc.transform(data_x_test_scaled)

#
data_x_train_scaled, data_x_test_scaled, data_y_train, data_y_test = \
    np.array(data_x_train_scaled), np.array(data_x_test_scaled), np.array(data_y_train), \
    np.array(data_y_test)

#
data_y_train = (data_y_train[:, 0], data_y_train[:, 1])
data_y_test = (data_y_test[:, 0], data_y_test[:, 1])

In [ ]:
# 0.0001 and 200 epochs --> good results

In [ ]:
# ==========================================================
# data prediction
# ==========================================================
input_layer = Input(shape=(data_x_train_scaled.shape[1]), name='Input_Layer')
common_path = Dense(units='128', activation='relu', name='First_Dense')(input_layer)
common_path = Dropout(0.3)(common_path)
common_path = Dense(units='128', activation='relu', name='Second_Dense')(common_path)
common_path = Dropout(0.3)(common_path)

first_output = Dense(units='1', name='First_Output__Last_Layer')(common_path)

second_output_path = Dense(units='64', activation='relu', name='Second_Output__First_Dense')(common_path)
second_output_path = Dropout(0.3)(second_output_path)
second_output = Dense(units='1', name='Second_Output__Last_Layer')(second_output_path)

#
model = Model(inputs=input_layer, outputs=[first_output, second_output])
print(model.summary())

#
optimizer = tf.keras.optimizers.SGD(learning_rate=0.00001)

#
model.compile(optimizer=optimizer,
              loss={'First_Output__Last_Layer': 'mse', 'Second_Output__Last_Layer': 'mse'},
              metrics={'First_Output__Last_Layer': tf.keras.metrics.RootMeanSquaredError(),
                       'Second_Output__Last_Layer': tf.keras.metrics.RootMeanSquaredError()})

In [ ]:
plot_model(model)

In [ ]:
# train the model
earlyStopping_callback = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=10,
                              verbose=1) 

history = model.fit(x=data_x_train_scaled, y=data_y_train, verbose=0, epochs=500, batch_size=10,
                    validation_split=0.3, callbacks=earlyStopping_callback)

In [ ]:
# display training history
print(history.history.keys())

In [ ]:
# summarize history for the first output rmse
plt.plot(history.history['First_Output__Last_Layer_root_mean_squared_error'])
plt.plot(history.history['val_First_Output__Last_Layer_root_mean_squared_error'])
plt.title('model\'s rmse for the first output')
plt.ylabel('rmse')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# summarize history for the second output accuracy
plt.plot(history.history['Second_Output__Last_Layer_root_mean_squared_error'])
plt.plot(history.history['val_Second_Output__Last_Layer_root_mean_squared_error'])
plt.title('model\'s rmse for the second output')
plt.ylabel('rmse')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
# summarize history for total loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Total Loss')
plt.ylabel('total loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# summarize history for the first output loss
plt.plot(history.history['First_Output__Last_Layer_loss'])
plt.plot(history.history['val_First_Output__Last_Layer_loss'])
plt.title('the first output Loss')
plt.ylabel('y1 loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# summarize history for the second output loss
plt.plot(history.history['Second_Output__Last_Layer_loss'])
plt.plot(history.history['val_Second_Output__Last_Layer_loss'])
plt.title('the second output Loss')
plt.ylabel('y2 loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
# Test the model
total_loss, first_output_loss, second_output_loss, first_output_rmse, second_output_rmse = np.round(model.evaluate(
    x=data_x_test_scaled, y=data_y_test, verbose=0), 3)
print(
    "Loss = {}, Y1_loss = {}, Y1_rmse = {}, Y2_loss = {}, Y2_rmse = {}".format(total_loss, first_output_loss,
                                                                             first_output_rmse, second_output_loss,
                                                                             second_output_rmse))

In [ ]:
LOG_DIR = f"{int(time.time())}"

In [ ]:
# ==========================================================
# Hyperparameter tuning using Keras Tuner
# ==========================================================
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=12,
    executions_per_trial=1,
    directory=LOG_DIR
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x=data_x_train_scaled,
             y=data_y_train,
             verbose=1,
             epochs=500,
             batch_size=10,
             validation_split=0.3,
             callbacks=[stop_early])

In [ ]:
tuner.results_summary()

In [ ]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The best result is as follow:
    dense_1_cp: {best_hps.get('dense_1_cp')},
    drouput_rate_1: {best_hps.get('drouput_rate_1')},
    dense_2_cp: {best_hps.get('dense_2_cp')},
    drouput_rate_2: {best_hps.get('drouput_rate_2')},
    dense_1_p2: {best_hps.get('dense_1_p2')},
    drouput_rate_3: {best_hps.get('drouput_rate_3')},
    learning_rate: {best_hps.get('learning_rate')}
""")

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model_tuned = tuner.hypermodel.build(best_hps)
history = model_tuned.fit(x=data_x_train_scaled, y=data_y_train, verbose=0, epochs=500, batch_size=10,
                    validation_split=0.3, callbacks=earlyStopping_callback)

In [ ]:
# Test the model
total_loss_tunned, first_output_loss_tunned, second_output_loss_tunned, first_output_rmse_tunned, second_output_rmse_tunned = np.round(model_tuned.evaluate(
    x=data_x_test_scaled, y=data_y_test, verbose=0), 3)
print(
    "Loss = {}, Y1_loss = {}, Y1_rmse = {}, Y2_loss = {}, Y2_rmse = {}".format(total_loss_tunned, first_output_loss_tunned,
                                                                             first_output_rmse_tunned, second_output_loss_tunned,
                                                                             second_output_rmse_tunned))

y1 Heating Load

y2 Cooling Load

In [ ]:
print(f'Results Before Tunning:\n Test Set RMSE: Heating Load:{np.round(first_output_rmse, 3)}, Cooling Load:{np.round(second_output_rmse, 3)}\n')
print(f'Results After Tunning:\n Test Set RMSE: Heating Load:{np.round(first_output_rmse_tunned, 3)}, Cooling Load:{np.round(second_output_rmse_tunned, 3)}\n')
print(f'{np.round((first_output_rmse - first_output_rmse_tunned)*100/first_output_rmse)}% Improvement in predicting Heating Load')
print(f'{np.round((second_output_rmse - second_output_rmse_tunned)*100/second_output_rmse)}% Improvement in predicting Cooling Load')